In [ ]:
import os
import pandas as pd
from pyvis.network import Network
import warnings

warnings.filterwarnings('ignore')

In [ ]:
kills = pd.DataFrame(columns=['universe', 'culprit', 'count', 'victims'])
deaths = pd.DataFrame(columns=['universe', 'victim', 'culprits', 'dependent', 'definitive'])
edges = pd.DataFrame(columns=['universe', 'culprit', 'victim', 'dependent', 'definitive'])

In [ ]:
for path, subdirs, files in os.walk('Universes'):
    for name in files:
        file = path + '\\' + name
        universe = file.split('\\')[1::2][0]
        
        with open(file, 'r') as f:
            lines = f.read().split('\n')
        
        for line in lines:
            victim = line.split('**')[1::2][0]
            culprits = line.split('_')[1::2]

            if '(dependent)' in line:
                dependent = True
            else:
                dependent = False
            
            if '(revived)' in line or '(undone)' in line:
                definitive = False
            else:
                definitive = True

            for i in range(len(culprits)):
                if culprits[i] == 'suicide':
                    culprits[i] = victim

                row = [universe, culprits[i], 1, victim]
                kills.loc[len(kills)] = row

                row = [universe, culprits[i], victim, dependent, definitive]
                edges.loc[len(edges)] = row
            
            if culprits == []:
                row = [universe, '', victim, dependent, definitive]
                edges.loc[len(edges)] = row

            row = [universe, victim, culprits, dependent, definitive]
            deaths.loc[len(deaths)] = row

In [ ]:
kills = kills.groupby(by=['universe', 'culprit']).aggregate({'count':sum, 'victims':', '.join})
kills = kills.reset_index().sort_values(by='count', ascending=False)
kills.drop_duplicates('universe')

In [ ]:
edges['culprit'] += " [" + edges['universe'] + "]"
edges.loc[edges['culprit'].str.startswith(' ['), 'culprit'] = ''
edges['victim'] += " [" + edges['universe'] + "]"
edges['color'] = pd.Series()

for i in range(len(edges)):
    if not edges.iloc[i]['definitive']:
        edges.iloc[i, 5] = '0_green'
    elif edges.iloc[i]['dependent'] and edges.iloc[i]['definitive']:
        edges.iloc[i, 5] = '1_purple'
    if not edges.iloc[i]['dependent'] and edges.iloc[i]['definitive']:
        edges.iloc[i, 5] = '2_crimson'

nodes = edges[['victim', 'color']]
nodes.sort_values(by=['victim', 'color'], inplace=True)
nodes.drop_duplicates('victim', keep='last', inplace=True)
victims = nodes['victim'].values
culprits = edges['culprit'].values

for culprit in culprits:
    if not culprit in victims:
        data = {'victim': [culprit], 'color': ['0_green']}
        df = pd.DataFrame(data)
        nodes = pd.concat([nodes, df])

nodes.sort_values(by='victim', inplace=True)
nodes.drop_duplicates('victim', keep='last', inplace=True)
nodes['color'] = nodes['color'].str.split('_').str[1]

In [ ]:
net = Network(height="100%", width="100%", bgcolor="#111111", font_color="#e8ac87", directed=True)
net.force_atlas_2based(overlap=1)
sources, targets = edges['culprit'], edges['victim']
edge_data = zip(sources, targets)

for i in range(len(nodes)):
    victim = nodes.iloc[i]['victim']
    color = nodes.iloc[i]['color']
    label = victim.split(" [")[0]

    if victim != '':
        net.add_node(victim, label=label, color=color)

for src, dst in edge_data:
    if src != '':
        net.add_edge(src, dst, color='#eb73b7')

neighbor_map = net.get_adj_list()

for node in net.nodes:
    node['title'] = node['label'] + "'s Victims :<br><br>" + "<br>".join(neighbor_map[node['id']])
    node['value'] = len(neighbor_map[node['id']])
    
net.inherit_edge_colors(False)
net.set_edge_smooth('dynamic')

net.save_graph('main.html')